<a href="https://colab.research.google.com/github/timofeyegorov/education/blob/main/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B2%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%BF%D0%B8%D0%BD%D0%B3_%D0%B8_%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D1%81%D0%B0%D0%B9%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Немного теории

In [ ]:
from IPython.display import Image # Библиотека для отображения картинок

display(Image(url='https://prowebscraper.com/blog/wp-content/uploads/2017/11/Web_Scraping_for_Non-Programmers.png', 
              width = 1400) # Задаем ширину картинки
       ) 

**Веб-скраппинг** - получения веб-данных путем извлечения их со страниц веб-ресурсов. Некоторые сайты предоставляют специальный API для программного доступа к своим данным.


**Парсинг** - анализ и обработка полученных данных.

Минусы веб-скраппинга.
- Он хрупкий (веб-страницы, которые вы очищаете, могут часто меняться).
- Это может быть запрещено (некоторые веб-приложения имеют политику против скрапинга).
- Это может быть медленным и экспансивным (если вам нужно забирать и пропускать много шума).
__________________________________________________
Вебскраппинг делится на статический и динамический.

**Статический** - получает статический код страницы. Игнорирует JavaScript. Можно выполнить с помощью библиотек requests и BeautifulSoup. С помощью requests отправляют запрос на получение информации с веб-страницы, а BeautifulSoup нужен, чтобы распарсить полученную html-разметку.

**Динамический** - использует фактический браузер и позволяет JavaScript работать. Иногда вам необходимо автоматизировать браузер, моделируя пользователя, чтобы получить необходимый контент. Используется библиотека Selenium. Она имтирует работе браузера, то можно нажимать на кнопки, заполнять формы.

# Скраппинг базы автомобилей с Авто.ру с BeautifulSoup

**Для чего нужна задержка для скраппинга.**

Если вы будете парсить более-менее серьезные сайты - можете столкнутся с защитой от парсинга.
Например, если парсить какой-то сайт достаточно длительное время и слать запросы к нему очень часто - этот сайт может вас заблокировать по ip и выдать капчу. Кроме того, это несет нагрузку на сайт, и хорошо бы поставить задержку, чтобы сильно не нагружть сервер.
Основной принцип обхода защиты от парсинга - ваш парсер должен вести себя так, как вел бы себя человек, зашедший на сайт. Для каждого сайта задержка подбирается индивидуально.

In [ ]:
from bs4 import BeautifulSoup  # импорт класса BeautifulSoup из модуля bs4 Для парсинга страниц
import requests                # Для отправки запросов к сайту
from requests import get       # Запрос на получение данных
from tqdm import tqdm
import urllib.request          # библиотка для скачивания изображений
import time
import random
import os
import pandas as pd

In [ ]:
# Ссылка на страницу, которую парсим, куда динамически подставляем номер страницы
url = f'https://auto.ru/novosibirsk/cars/all/?page=1'
response = get(url)         # Получаем статус ответа сервера, 200 - все ок

response.encoding                                             # Смотрим какая кодировка используется
content = response.content                                    # Получаем байтовое представление данных для нетекстовых запросов
html_soup = BeautifulSoup(content.decode('utf-8','ignore'))   # Декодируем данные, игнорируя ошибки в символах

In [ ]:
start_time = time.time()        # Засекаем время выполнения кода в ячейке
cars = []                       # Список для хранения спарсенных данных для каждого объявления
count = 1                       # Счетчик страниц сайта, начинаем извлекать данные с первой страницы

while count <= 5:               # Устанавливаем максимальное количество страниц с которых будем собирать данные
    print(f'Страница {count}')  # Информация для вывода о том, какая страница сейчас парсится

    # Ссылка на страницу, которую парсим, куда динамически подставляем номер страницы
    url = f'https://auto.ru/novosibirsk/cars/all/?page={count}'
    response = get(url)         # Получаем статус ответа сервера, 200 - все ок
    
    response.encoding                                             # Смотрим какая кодировка используется
    content = response.content                                    # Получаем байтовое представление данных для нетекстовых запросов
    html_soup = BeautifulSoup(content.decode('utf-8','ignore'))   # Декодируем данные, игнорируя ошибки в символах

    #html_soup = BeautifulSoup(response.text, 'html.parser') - заменяет предыдущие 3 строки кода, если нет проблем с кодировкой


    cars_data = html_soup.find_all('div', class_ = 'ListingItem-module__description') # Находим все div блоки с классом 'ListingItem-module__main'
                                                                               # Метод возвращает список, где каждый div блок - это отдельное объявление

    if cars_data != []:                                                        # Проверяем, что данные есть
      cars.extend(cars_data)                                                   # Добавляем в список данных данные всех объявлений с текущей страницы
      print(f'Со страницы спарсено {len(cars_data)} объявлений, всего объявлений спарсено: {len(cars)}')

      # Генерируем задержку до следующего запроса от 1 до 11 секунд
      value = random.random()                                               # Генерируем случайное число от 0 до 1
      time_sleep = 1 + value*10                                             # Добавляем случайное число к единице
      print(f'Задержка до следующего запроса: {round(time_sleep, 1)} сек')
      time.sleep(time_sleep)                                                # Откладываем исполнение кода на time_sleep секунд
      print('--------------------------------')

    else:
      print('empty')                                                        # Если данные не справились - прекращаем цикл
      break  
    count += 1                                                              # Увеличиваем счетчик на 1 (переходим к парсингу следующей страницы)

end_time = time.time() - start_time                                         # Считаем сколько времени ушло на парсинг
print()
print(f'Время извлечения данных: {round(end_time/60, 1)} минут')

In [ ]:
count = 0       # Обнуляем счетчик, будем извлекать нужную информацию с нулевого элемента
data = []       # Список для хранения фильтрованной информации по каждому объявлению
skip_ads = 0    # Счетчик пропущенных объявления

# Задаем, какое кол-во объявлений спарсим
while count <= (len(cars)-1):
  try:                                                                            # Применяем try - except, чтобы обработать ошибку, когда не указана цена авто
    info = cars[int(count)]                                                       # Получаем блок div по объявлению
    title = info.find('a', {'class':'Link ListingItemTitle-module__link'}).text   # Находим тег 'a' с классом 'ListingItem-module__main', извлекаем текстовую информацию
    price = info.find('div', {'class':'ListingItemPrice-module__content'}).text   # Находим тег div с классом 'ListingItem-module__main', извлекаем текстовую информацию

    other_info = info.find_all('div', {'class':'ListingItemTechSummaryDesktop__cell'}) # Находим ВСЕ теги div с классом 'ListingItemTechSummaryDesktop__cell' (их 5 штук), метод возвращает список

    # Для каждого из пяти тегов div с классом 'ListingItemTechSummaryDesktop__cell' получаем текстовую информацию
    characteristics = str(other_info[0].get_text())
    transmission = str(other_info[1].get_text())
    body = str(other_info[2].get_text())
    drive = str(other_info[3].get_text())
    color = str(other_info[4].get_text())

    # Выводим данные которые спарсили для каждого объявления
    print(count, '. ', title, ' - ', price, ' - ', characteristics, ' - ', transmission, ' - ', body, ' - ', drive, ' - ', color, sep = '')

    temp_list = [title, price, characteristics, transmission, body, drive, color] # Во временный список добавляем все спарсенные данные по текущему объявлению
    data.append(temp_list)                                                        # Добавляем данные по текущему объявлению в дата для дальнейшего преобразования в пандас ДатаФрейм
  except AttributeError:
    skip_ads +=1                                                                # Считаем кол-во пропущенных объявлений
    print(count)
  count += 1                                                                    # Увеличиваем счетчик для перехода к парсингу следующего объявления
print()
print(f'Пропущено {skip_ads} объявлений')  

In [ ]:
columns = ['title', 'price', 'characteristics', 'transmission', 'body', 'drive', 'color']
df = pd.DataFrame(data, columns=columns)

In [ ]:
df

# Скраппинг изображений с BeautifulSoup

In [ ]:
link_root = f'http://www.motorpage.ru'                                          # Корневая ссылка на сайт, откуда будем парсить объявления
response = get(link_root + '/media/photos.html')                                # Получение информации о галерее изображений
html_soup = BeautifulSoup(response.text)                                        # Получаем разметку html-страницы для последующей обработки

cars_data = html_soup.find_all('a', {'class': 'col-xs-12 col-sm-6 zero-padding'})
cars_data

In [ ]:
cars_data[:3]

In [ ]:
brand_links = [link['href'] for link in cars_data]   
brand_links[:3]

In [ ]:
brand_names = [link[1:link.rfind('/')].capitalize() for link in brand_links] 

In [ ]:
('acura').capitalize()

In [ ]:
brand_dict = {k:v for k, v in zip(brand_names, brand_links)}                    # Создаем словарь, где ключ - название бренда, значение - окончание ссылки на страницу с фото
number_cars = html_soup.find_all('figcaption', {'class': 'caption-photos'})     # По каждому авто парсим информацию по количеству фото
cars_values = [int(cars.text) for cars in number_cars]        

In [ ]:
brand_numbers = list(zip(brand_names, cars_values)) 
brand_numbers[:3]

In [ ]:
# Соберем информацию в текстовом виде о наличии фотографий для каждой марки авто

link_root = f'http://www.motorpage.ru'                                          # Корневая ссылка на сайт, откуда будем парсить объявления
response = get(link_root + '/media/photos.html')                                # Получение информации о галерее изображений
html_soup = BeautifulSoup(response.text)                                        # Получаем разметку html-страницы для последующей обработки

cars_data = html_soup.find_all('a', {'class': 'col-xs-12 col-sm-6 zero-padding'})  # Находим блоки с информацией о количестве фото по каждой марке авто

brand_links = [link['href'] for link in cars_data]                              # Получение окончания ссылки для каждого бренда, например, Acura/photos.html (полный адрес: http://www.motorpage.ru/Acura/photos.html)
brand_names = [link[1:link.rfind('/')].capitalize() for link in brand_links]    # Получение имени каждого бренда из окончания ссылки

brand_dict = {k:v for k, v in zip(brand_names, brand_links)}                    # Создаем словарь, где ключ - название бренда, значение - окончание ссылки на страницу с фото
number_cars = html_soup.find_all('figcaption', {'class': 'caption-photos'})     # По каждому авто парсим информацию по количеству фото
cars_values = [int(cars.text) for cars in number_cars]                          # Получаем тексотовую информацию о количестве фото по каждому авто из number_cars

brand_numbers = list(zip(brand_names, cars_values))                             # Создаем список кортежей из парных элементов, где первый элемент - бренд авто, второй - количество фотографий этого бренда
brand_numbers.sort(key = lambda x: x[1], reverse = True)                        # Делаем сортировку списка по количеству фото (второму элементу кортежа)

# Выводим информацию по количеству фотографий для каждого бренда
num_ = 1
for i, j in (brand_numbers):
  print(f'{num_}. Для бренда {i} можно скачать {j} изображений')
  num_ += 1

In [ ]:
# Выбор брендов для скачивания изображений
selected_brands = [brand_name for brand_name, brand_num in brand_numbers[79:82]]
selected_brands

In [ ]:
['country',	'Сколько_вам_лет',	'В_какой_сфере_сейчас_работаете',	'Ваш_средний_доход_в_месяц',	'Рассматриваете_ли_в_перспективе_платное_обучение_профессии_Разработчик_Искусственного_Интеллекта',	'Сколько_времени_готовы_выделить_на_обучение_в_неделю']:

In [ ]:
country, age, job, earnings, training, time

In [ ]:
data_path = './cars_images/'          # Указание пути для сохранения данных
os.makedirs(data_path, exist_ok=True) # Создание папки для данных, если её не существует

In [ ]:
# Сначала спарсим все ссылки на фотографии
start_time = time.time()                                                        # Засекаем время выполнения кода в ячейке
count = 1                                                                       # Счетчик страниц сайта, начинаем извлекать данные с первой страницы
link_dict = {}                                                                  # Словарь для хранения ссылок на фото для брендов {бренд: ссылки}

for brand_name in selected_brands:                                              # Итерация по рассматриваемым брендам
  print(f'{count}. Получение ссылок для бренда: {brand_name}')                  # Информация о процессе
  

  current_link = link_root + brand_dict[brand_name]                             #  Получаем текущую ссылку (пример: http://www.motorpage.ru/Hummer/photos.html)
  
  response = requests.get(current_link)                                         # Делаем запрос на сайт и получаем ответ
  html_soup = BeautifulSoup(response.text)                                      # Получаем html для обработки информации

  model_cards = html_soup.findAll('a', {'class': "col-xs-12 col-sm-4 col-md-3"}) # По тегу и классу получаем блоки с информацией по каждой модели выбранной марки
  model_links = [model_card['href'] for model_card in model_cards]              # Получение ссылок на изображения по каждой марке каждого бренда
  brand_links = []                                                              # Хранилище для ссылок на изображения машин данного бренда

  for model_link in tqdm(model_links):                                          # итерация по всем ссылкам на изображения с моделями
      direct_link = link_root + model_link                                      # Получаем ссылку, пример: http://www.motorpage.ru/Hummer/H2/last/photos/
      response = requests.get(direct_link)                                      # Делаем запрос на сайт
      html_soup = BeautifulSoup(response.text)                                  # Получаем объект для обработки информации
      picture_frames = html_soup.findAll('img', {'title': True})                # Собираем все блоки с тегом img, {'title': True} поможет найти только блоки с изображениями, где у которых есть заголовок
      picture_links = [frame['src'] for frame in picture_frames]                # Сбор ссылок на исходные изображенения 
      brand_links += picture_links                                              # Добавляем в список brand_links все ссылки на изображения по текущей модели авто   
  link_dict[brand_name] = brand_links                                           # Добавляем в словарь link_dict ключ бренда авто и список ссылок на изображения в качестве значения


  # Генерируем задержку до следующего запроса от 1 до 4 секунд
  value = random.random()                                                       # Генерируем случайное число от 0 до 1
  time_sleep = 1 + value*3                                                      # Добавляем случайное число к единице
  print(f'Задержка до следующего запроса: {round(time_sleep, 1)} сек')
  time.sleep(time_sleep)                                                        # Откладываем исполнение кода на time_sleep секунд
  count +=1
  print('--------------------------------')

end_time = time.time() - start_time                                             # Считаем сколько времени ушло на парсинг
print()
print(f'Время извлечения данных: {round(end_time/60, 1)} минут')  

In [ ]:
# Скачиваем изображения

start_time = time.time()                                                        # Засекаем время выполнения кода в ячейке
count = 1                                                                       # Счетчик страниц сайта, начинаем извлекать данные с первой страницы

for brand_name, link_list in link_dict.items():                                 # В цикле берем бренд и список ссылок на его изображение
  print(f'{count}. Скачивание изображений для бренда: {brand_name}')            # Выводим инфо процесса
  os.makedirs(data_path + brand_name, exist_ok=True)                            # Создание папки бренда, если её не существует
  dir_path = data_path + brand_name                                             # Указываем текущую директорию, куда сохраняем изображения

  for idx, link in enumerate(tqdm(link_list)):                                  # Проходим по каждой ссылке
    urllib.request.urlretrieve(link, f"{dir_path}/{'0'*(4-len(str(idx)))}{idx}_{brand_name}.jpg") # Сохраняем изображение

  count +=1
  print()
  print('--------------------------------')

end_time = time.time() - start_time                                             # Считаем сколько времени ушло на парсинг
print()
print(f'Время скачивания: {round(end_time/60, 1)} минут')      

# Получение данных с kinopoisk.ru через API

**Что такое API**

API (Application programming interface) — это контракт, который предоставляет программа. «Ко мне можно обращаться так и так, я обязуюсь делать то и это».

Подробнее: https://habr.com/ru/post/464261/

https://kinopoisk.cloud/

In [ ]:
# Получаем и изучаем json файл

api_token='Ваш Токен'   # Токен, доступный после регистрации

url = f'https://api.kinopoisk.cloud/movies/{1143242}/token/Ваш Токен'    # Url для запроса данных с сайта
request = get(url)              # Отправляем запрос, получаем ответ
data = request.json()           # Преобразуем ответ в json
data

In [ ]:
# Пример отсутствия фильма по id

url = f'https://api.kinopoisk.cloud/movies/{1143244}/token/c9972b77e92bbc443c28279e1a29ec58'
request = get(url) # Отправляем запрос, получаем ответ
data = request.json() 
data

In [ ]:
start_time = time.time() # Засекаем время выполнения кода в ячейке

films_data = [] # Список для хранения данных по всем фильмам

# Названия ключей json файла, к кторым будем обращаться для получения нужных данных
params = ['title', 'id_kinopoisk', 'rating_imdb', 'rating_kinopoisk', 'year', 'actors', 'budget', 'composers', 'directors', 'fees_world', 'genres', 'description']
skip_films = 0 # Счетчик пропущенных id
count = 1 # Счетчик итераций

for movie_number in range(1143242, 1143242+90):  # Перебираем id фильмов
    try:
        url = f'https://api.kinopoisk.cloud/movies/{movie_number}/token/c9972b77e92bbc443c28279e1a29ec58' # Передаем id фильма в url
        request = get(url)  # Отправляем запрос, получаем ответ
        data = request.json()  # Получаем json файл

        print(f"{count}. Получение информации для фильма '{data['title']}'") # Информация о процессе
        temp_list = []                                    # Создаем временный список, куда вносим данные по текущему фильму
        for param in params:                              # Проходимся по нужным нам ключам в json файле
           temp_list.append(data[param])                  # Добавляем данные во временный список
        temp_list.append(data['collapse']['duration'])    # Отдельно добавляем информацию по длительности фильма
        films_data.append(temp_list)                      # В список films_data добалвяем как элемент временный список temp_list
        count += 1

    except KeyError:                                      # В случае, если фильма нет, выводим об этом сообщение
        print(f'{count} фильм с id {movie_number} отсутствует в базе')
        count += 1
        skip_films += 1 

end_time = time.time() - start_time                                         # Считаем сколько времени ушло на парсинг
print()
print(f'Время извлечения данных: {round(end_time/60, 1)} минут')
print(f'Пропущено {skip_films} id')   

In [ ]:
df = pd.DataFrame(films_data, columns=['title', 'id', 'Рейтинг IMDB', 'Рейтинг Кинопоиск', 'Год', 'Актёры', 'Бюджет', 'Композиторы', 'Режиссёр', 'Сборы', 'Жанры', 'Описание', 'Длительность']) # создаем датафрейм

In [ ]:
df